# Genty

**A test helper that promotes generative testing, where one test can run over a variety of inputs.**

`pip install genty`

Available on GitHub: https://github.com/box/genty

## Why?

- Have you ever written several tests that look very similar and wish they could share code?
- Have you ever written tests that repeat the same assertions for different data sets?

*How maintainable were those tests?*

**Genty lets you separate your test code from test data.**

Writing tests using genty results in:

- DRY test code
- maintainable test code
- a clear separation of test logic and test data

Using genty promotes writing that test for that edge case that you might otherwise have skipped.

## Examples

Imagine we have a class library of shapes. Each shape can calculate its own `area()`.

In [1]:
# Quick and simple test runner
class Runner(object):
    def __init__(self, klass):
        self._klass = klass
    
    def run(self):
        test_class = self._klass()
        for test in (t for t in dir(test_class) if t.startswith('test_')):
            passed = False
            try:
                getattr(test_class, test)()
                passed = True
            except:
                pass
            print test + ' ' + ('OK' if passed else 'F')

In [2]:
import math

class Shape(object):
    def area(self):
        raise NotImplemented

class Rectangle(Shape):
    def __init__(self, length, width):
        self._length = length
        self._width = width

    def area(self):
        return self._length * self._width

class Square(Rectangle):
    def __init__(self, length):
        super(Square, self).__init__(length, length)
    
class Circle(Shape):
    def __init__(self, radius):
        self._radius = radius

    def area(self):
        return self._radius ** 2 * math.pi

Testing the `area()` calculations of each shape isn't too difficult.

In [3]:
class TestShapes(object):
    def test_square_area(self):
        assert Square(3).area() == 9

    def test_rectangle_area(self):
        assert Rectangle(3, 5).area() == 15

    def test_funky_rectangle_area(self):
        assert Rectangle(3, 0).area() == 0

    def test_circle_area(self):
        assert Circle(2).area() == 4*math.pi

In [4]:
Runner(TestShapes).run()

test_circle_area OK
test_funky_rectangle_area OK
test_rectangle_area OK
test_square_area OK


However, we do see plenty of code duplication.

* What would happen if we refactored the code under test? Lots of test code changes as well.
* New shape? Several new test methods as well.
* New method in the code under test? **Test code explosion!**

In [5]:
class TestShapes2(object):
    def test_shape_area(self):
        assert Square(3).area() == 9
        assert Rectangle(3, 5).area() == 15
        assert Rectangle(3, 0).area() == 0
        assert Circle(2).area() == 4*math.pi

In [6]:
Runner(TestShapes2).run()

test_shape_area OK


Wow! That's a lot less code. But it's also a lot less test feedback.

Let's try the same thing using genty.

In [7]:
from genty import genty, genty_dataset

@genty
class TestShapes3(object):
    @genty_dataset(
        (Square(3), 9),
        (Rectangle(3, 5), 15),
        (Rectangle(3, 0), 0),
        (Circle(2), 4 * math.pi),
    )
    def test_shape_area(self, shape, expected_area):
        assert shape.area() == expected_area

In [8]:
Runner(TestShapes3).run()

test_shape_area(<__main__.Circle object at 0x1065273d0>, 12.5663706144) OK
test_shape_area(<__main__.Rectangle object at 0x10651c550>, 15) OK
test_shape_area(<__main__.Rectangle object at 0x106527310>, 0) OK
test_shape_area(<__main__.Square object at 0x10651c5d0>, 9) OK


Cool! Our test data is separated from our test logic, and we get detailed test feedback.

## Features

### Named datasets

Genty will automatically name your generated tests for you, but you can specify the names directly.

In [9]:
from genty import genty, genty_dataset

@genty
class TestShapes3(object):
    @genty_dataset(
        square_3x3=(Square(3), 9),
        recatngle_3x5=(Rectangle(3, 5), 15),
        rectangle_3x0=(Rectangle(3, 0), 0),
        circle_radius_2=(Circle(2), 4 * math.pi),
    )
    def test_shape_area(self, shape, expected_area):
        assert shape.area() == expected_area

In [10]:
Runner(TestShapes3).run()

test_shape_area(circle_radius_2) OK
test_shape_area(recatngle_3x5) OK
test_shape_area(rectangle_3x0) OK
test_shape_area(square_3x3) OK


### Optional/named parameters

Genty fills test parameters from `genty_dataset`s, but you might prefer to give some test methods parameters with default values.

In that case, you can use `genty_args`. Simply pass `genty_args` parameters as you would pass to the test method.

In [11]:
from genty import genty_args

@genty
class TestComplex(object):
    @genty_dataset(
        default_case=(0, 1),
        limit_case=(999, 1000),
        error_case=genty_args(-1, -1, is_something=False),
    )
    def test_complex(self, value1, value2, optional_value=None, is_something=True):
        assert is_something

In [12]:
Runner(TestComplex).run()

test_complex(default_case) OK
test_complex(error_case) F
test_complex(limit_case) OK


### Repeating a test

`genty_repeat` lets you run a test multiple times.

In [13]:
from genty import genty_repeat

@genty
class TestSomething(object):
    @genty_repeat(5)
    def test_something(self):
        pass

In [14]:
Runner(TestSomething).run()

test_something() iteration_1 OK
test_something() iteration_2 OK
test_something() iteration_3 OK
test_something() iteration_4 OK
test_something() iteration_5 OK


### Deferred data generation

Normally, data passed to `genty_dataset` needs to be known at test definition time.

For data that might not be known until runtime, there is `genty_dataprovider`, which defers data generation to just before the test is run.

Simply pass in a helper method that will return a tuple; the tuple will be unpacked and sent as `*args` to the test method.

In [15]:
from datetime import datetime
from genty import genty_dataprovider

@genty
class TestSomething2(object):
    def _helper(self):
        return datetime.utcnow(), 'foo'

    @genty_dataprovider(_helper)
    def test_something(self, date, name):
        pass

In [16]:
Runner(TestSomething2).run()

test_something__helper OK


`genty_dataprovider` helper methods can in turn be decorated with `@genty_dataset`, allowing the helper to be parametrized.

In [17]:
@genty
class TestSomething3(object):
    @genty_dataset('foo', 'bar')
    def _helper(self, name):
        return datetime.utcnow(), name

    @genty_dataprovider(_helper)
    def test_something(self, date, name):
        pass

In [18]:
Runner(TestSomething3).run()

test_something__helper('bar') OK
test_something__helper('foo') OK


## Summary

- `@genty`/`@genty_dataset` let you easily parametrize your tests, separating test data from test logic.
- `@genty_repeat` lets you run the same test multiple times.
- `@genty_args` lets you specify args and kwargs just as you would to the test method.
- `@genty_dataprovider` lets you defer the creation of test data until runtime.

This notebook is available for download: http://opensource.box.com/genty/Genty.ipynb

Its content is licensed under the [Apache License 2.0](https://www.apache.org/licenses/LICENSE-2.0).